# Determinante via Cholesky

Calcular o determinante de uma matriz M usando propriedades de algebra linear, utilizando a **Decomposição de Cholesky**.

---

## Teoremas e Definições
> **Definição: Matriz Positivamente SemiDefinida**
> Uma matriz simétrica $S in \real^mxd$ é dita **Positivamente Semidefinida** se, para todo vetor  não nulo $z \in \real^d $, o seguinte for verdadeiro:
> $$z^T S z \ge 0$$
> Sempre é possível encontrar uma matriz $C$ tal que $S = C^T C$.
> det(S) = det(C)^2

> **Decomposição de Cholesky**
>
> Se uma matriz simétrica $S$ é **Positivamente Definida** (ou Semidefinida), ela pode ser decomposta unicamente como:
> $$S = L L^T$$
> Onde $L$ é uma **matriz triangular inferior** com diagonal positiva.

> **Determinante**
>
> O determinante da matriz original é o quadrado do produto da diagonal de $L$:
> $$\det(S) = \left( \prod_{i=0}^{N-1} L_{ii} \right)^2$$

---

##  (Caso 3x3)

O objetivo é encontrar $L$ tal que:

$$
\begin{bmatrix}
S_{00} & S_{01} & S_{02} \\
S_{10} & S_{11} & S_{12} \\
S_{20} & S_{21} & S_{22}
\end{bmatrix}
=
\begin{bmatrix}
L_{00} & 0 & 0 \\
L_{10} & L_{11} & 0 \\
L_{20} & L_{21} & L_{22}
\end{bmatrix}
\times
\begin{bmatrix}
L_{00} & L_{10} & L_{20} \\
0 & L_{11} & L_{21} \\
0 & 0 & L_{22}
\end{bmatrix}
$$

###  Passo 1: A Primeira Coluna (Coluna 0)
Extraímos a raiz da diagonal e dividimos o restante:
* $L_{00}^2 = S_{00} \implies L_{00} = \sqrt{S_{00}}$
* $L_{10} L_{00} = S_{10} \implies L_{10} = S_{10} / L_{00}$
* $L_{20} L_{00} = S_{20} \implies L_{20} = S_{20} / L_{00}$

###  Passo 2: A Segunda Coluna (Coluna 1)
Subtraímos a influência da coluna anterior:
* $L_{10}^2 + L_{11}^2 = S_{11} \implies L_{11} = \sqrt{S_{11} - L_{10}^2}$
* $L_{20}L_{10} + L_{21}L_{11} = S_{21} \implies L_{21} = (S_{21} - L_{20}L_{10}) / L_{11}$

###  Passo 3: A Última Coluna (Coluna 2)
A diagonal final absorve tudo o que restou:
* $L_{20}^2 + L_{21}^2 + L_{22}^2 = S_{22} \implies L_{22} = \sqrt{S_{22} - (L_{20}^2 + L_{21}^2)}$

---

## Vetorização
Da definição de produto de matrizes, podemos usar a seguinte fórmula geral para calcular os elementos de L recursivamente:

$$L_{ij} = \frac{1}{L_{jj}} \left( S_{ij} - \sum_{k=0}^{j-1} L_{ik} L_{jk} \right)$$

A soma pode ser substituída por um produto escalar entre as linhas apropriadas de L:

```python
# Em vez de um loop lento 'for i in range...':
# Calculamos a "Correção" para toda a coluna abaixo da diagonal de uma vez:
correction = np.dot(L[j+1:, :j], L[j, :j])

# Aplicamos o feitiço em bloco:
L[j+1:, j] = (S[j+1:, j] - correction) / L[j, j]

In [2]:
import numpy as np
import time
import matplotlib.pyplot as plt

def time_function(f):
    def wrap(*args):
        start = time.time()
        ret = f(*args)
        end = time.time()
        return ret, end - start
    return wrap

@time_function
def quick_det_vectorized(A):
    S = A @ A.T
    N = S.shape[0]
    L = np.zeros((N, N))

    for j in range(N):
        s = S[j, j] - np.dot(L[j, :j], L[j, :j])

        if s <= 0: s = 1e-10
        L[j, j] = np.sqrt(s)

        correction = np.dot(L[j+1:, :j], L[j, :j])

        L[j+1:, j] = (S[j+1:, j] - correction) / L[j, j]

    det = np.prod(np.diag(L))**2
    return det

tempos_minha = []
tempos_numpy = []
tamanhos = range(10, 2000, 100) #

for N in tamanhos:
    M_E = np.random.rand(N, N)
    S_ref = M_E @ M_E.T

    det, tempo_meu = quick_det_vectorized(M_E)
    tempos_minha.append(tempo_meu)

    start = time.time()
    np.linalg.det(S_ref)
    end = time.time()
    tempos_numpy.append(end - start)

plt.figure(figsize=(10, 6))
plt.plot(tamanhos, tempos_minha, label='Minha ', linewidth=2)
plt.plot(tamanhos, tempos_numpy, label='NumPy ', linestyle='--')
plt.xlabel('Tamanho da Matriz (N)')
plt.ylabel('Tempo (segundos)')
plt.title('Comparação de Performance: Python Otimizado vs NumPy')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

ModuleNotFoundError: No module named 'numpy'